In [1]:
import pandas as pd
from pathlib import Path
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_csv(Path("Sample GPT tweets.csv"))

## getting rid of rows that has nothing to do with ChatGPT based on the text only.

In [3]:
chat_gpt = ["ChatGPT ", "chatgpt" , "#chatGPT" , "#chatgpt"]
text_cleaned_df = df.loc[df.text.str.contains('|'.join(chat_gpt),case =True)]
text_cleaned_df

,user_name,text,user_location,result,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,Unnamed: 13
0,Math_Maniac,ChatGPT does ascii art\n#ChatGPT https://t.co/...,"Decatur, GA",NaN,I work with a number of tools. Right now it's ...,2020-07-20 14:24:50+00:00,467,1233,49216,False,2023-04-02 13:44:07+00:00,['ChatGPT'],Twitter Web App,chatgpt
1,SmashKoders,#ChatGPT is funny sometimes 😅 https://t.co/cSa...,Mumbai,NaN,My life changed while debugging code 😊,2022-06-18 04:34:18+00:00,8,126,239,False,2023-04-02 13:43:46+00:00,['ChatGPT'],Twitter Web App,NaN
2,Andreas Illenseer,Let’s put it in graphic terms. Drawing a #Unic...,"Mainz, Deutschland",NaN,Specialized in Regulatory & Digital Intelligen...,2017-10-31 19:35:24+00:00,63,188,88,False,2023-04-02 13:43:36+00:00,['Unicorn'],Twitter for iPhone,NaN
4,The Media Scoop,Italian data protection agency placed an immed...,India,NaN,Independent news aggregator & YouTuber at The ...,2023-03-07 01:51:53+00:00,0,37,0,False,2023-04-02 13:42:56+00:00,NaN,Twitter Web App,NaN
5,Game Development Daily News,Will your games be marketed by ChatGPT? https:...,NaN,NaN,Your source for daily news about game developm...,2020-09-03 15:30:02+00:00,2525,724,1108,False,2023-04-02 13:42:54+00:00,"['gamedev', 'gamedevelopment', 'indiedev', 'AI...",Twitter Web App,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Most Bay▶️,"RiverRise(.)com SOLD for $2,288 -BuyDomains, A...",JakartaX.com,NaN,Investor in #Crypto & #DomainNames. Selling: h...,2012-09-15 17:30:34+00:00,2343,4338,15594,False,2023-04-02 13:14:06+00:00,NaN,Twitter Web App,NaN
95,fracksix,#ChatGPT cannot save us. https://t.co/tmAMM5zdzu,NaN,NaN,NaN,2023-03-07 18:17:30+00:00,3,28,0,False,2023-04-02 13:13:58+00:00,['ChatGPT'],Twitter for iPhone,NaN
97,Sundar Narasimhan,Great discussion at @theallinpod esp. regardin...,NaN,NaN,NaN,2012-02-19 20:32:50+00:00,99,1058,1175,False,2023-04-02 13:13:37+00:00,NaN,Twitter for Android,NaN
98,Most Bay▶️,"ClientFinancing(.)com SOLD for $2,288 -BuyDoma...",JakartaX.com,NaN,Investor in #Crypto & #DomainNames. Selling: h...,2012-09-15 17:30:34+00:00,2343,4338,15594,False,2023-04-02 13:12:06+00:00,NaN,Twitter Web App,NaN


In [4]:
bot_df = text_cleaned_df.copy()

## Filtering and Cleaning the bot or automated tweets from data 

In [5]:
#creating dataset with variables that can be used as potential indication of bot tweets 
bot_df = bot_df[["user_name", "user_created" , "user_followers" , "user_friends" , "user_favourites", "user_verified" , "source"]]


In [6]:
#account verification cleaning
bot_df["user_verified"].value_counts()

False    70
True      5
Name: user_verified, dtype: int64

In [7]:
bot_df["user_verified"] = bot_df["user_verified"].astype(bool)

In [8]:
bot_df["user_verified"] = bot_df["user_verified"].apply(lambda x: str(x))

In [9]:
verification_filtered_df =bot_df.copy()
verification_filtered_df = verification_filtered_df[verification_filtered_df
                                                    ["user_verified"].str.contains("False")==True]

In [10]:
#source cleaning
source_filtered_bot_df = verification_filtered_df.copy()
source_filtered_bot_df = verification_filtered_df[~verification_filtered_df.source.str.contains("Tw")]
source_filtered_bot_df

,user_name,user_created,user_followers,user_friends,user_favourites,user_verified,source
16,1827 Marketing News,2019-11-29 23:36:12+00:00,191,1,3620,False,Missinglettr
26,SaasHoles,2020-07-14 18:46:41+00:00,59,88,198,False,Constant Contact - Social Posts
29,The Baby Care Foundation (TBCF),2019-07-20 05:36:24+00:00,270,28,72,False,Metricool
30,Neoskeptics,2020-04-23 06:03:16+00:00,46,1,23,False,Hootsuite Inc.
59,keith parnell,2007-05-02 18:17:39+00:00,5969,4457,6677,False,Nelio Content
84,Amperly AI insights,2022-11-12 17:52:18+00:00,199,53,794,False,SocialChamp IO


In [11]:
# creating new column(bot) to indicate what most likely to be bot account (1 as bot and 0 as human)
soure_filtered_bot_df = source_filtered_bot_df.set_index("user_name")
bot_list = soure_filtered_bot_df.index.values.tolist()


In [12]:
text_cleaned_df["bot"] = 0

C:\Users\alber\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
for key, row in text_cleaned_df.iterrows():
    for y in bot_list:
        if row[0] == y:
            text_cleaned_df.loc[key, "bot"] = 1
       

C:\Users\alber\anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
text_cleaned_df["bot"].value_counts()


0    69
1     6
Name: bot, dtype: int64

## Roberta Polarity Test

In [15]:
sample_tweet = text_cleaned_df["text"].values[35]
sample_tweet

"#ChatGPT #OpenAI I'm all for developing humanity. But at what cost. Are we ready for a superintelligent being that sees us as a challenge to limited resources? \nIt isn't sci-fi anymore, it is dangerously close. #showerthoughts #chatgpt5"

In [16]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(roberta)
model = AutoModelForSequenceClassification.from_pretrained(roberta)


In [17]:
result={}
text_list = []
name_list = []

In [18]:
def polarity_scores(sample_tweet):
    encoded_tweet = tokenizer(sample_tweet, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_value = {'roberta_negative': scores[0],
                    'roberta_neutral': scores[1],
                    'roberta_positive': scores[2]
                   }
    return scores_value

for index, row in text_cleaned_df.iterrows():
    text = row["text"]
    name_list.append(row["user_name"])
    result[text] = polarity_scores(text)

In [19]:
roberta_analysis_df = pd.DataFrame.from_dict(result)
roberta_analysis_df = roberta_analysis_df.T

In [20]:
roberta_analysis_df['user_name'] = name_list

In [21]:
roberta_analysis_df = roberta_analysis_df.reset_index()

In [22]:
roberta_analysis_df = roberta_analysis_df.drop(["index"], axis = 1 )

In [23]:
roberta_analysis_df = roberta_analysis_df[["user_name", "roberta_negative" , "roberta_neutral" , "roberta_positive"]]
roberta_analysis_df.head()

,user_name,roberta_negative,roberta_neutral,roberta_positive
0,Math_Maniac,0.035246,0.910445,0.054309
1,SmashKoders,0.031501,0.375222,0.593278
2,Andreas Illenseer,0.006352,0.114942,0.878707
3,The Media Scoop,0.338151,0.620923,0.040926
4,Game Development Daily News,0.036832,0.884951,0.078217
